## 1. Definición de las tablas DLT
Siguiendo la arquitectura medallón

In [0]:
import dlt
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StringType, DoubleType, LongType

# Esquema de los datos
schema = StructType() \
    .add("timestamp", LongType()) \
    .add("crypto", StringType()) \
    .add("price_usd", DoubleType())

# Tabla bruta (raw)
@dlt.table(
    name="raw_crypto_prices",
    comment="Raw cryptocurrency price data ingested from JSON files.",
)
def raw_crypto_prices():
    return (
        spark.readStream
        .schema(schema)
        .json("/tmp/crypto_dlt/")
    )

# Tabla transformada (processed)
@dlt.table(
    name="processed_crypto_prices",
    comment="Cleaned and transformed cryptocurrency price data.",
)
def processed_crypto_prices():
    return (
        dlt.read("raw_crypto_prices")
        .withColumn("event_time", col("timestamp").cast("timestamp"))
        .select("event_time", "crypto", "price_usd")
    )

# Tabla agregada (aggregate)
@dlt.table(
    name="aggregated_crypto_prices",
    comment="Aggregated cryptocurrency prices over a 1-minute window.",
)
def aggregated_crypto_prices():
    return (
        dlt.read("processed_crypto_prices")
        .groupBy(
            col("crypto"),
            col("event_time").alias("timestamp")
        )
        .avg("price_usd")
        .withColumnRenamed("avg(price_usd)", "avg_price_usd")
    )


## 2. Query a las tablas DLT

### 2.1 Bronze Layer(raw data)

In [0]:
%sql
SELECT * FROM taller_de_big_data_uni.cryptos.processed_crypto_prices
WHERE crypto = "bitcoin"
;

In [0]:
%sql
SELECT * FROM `taller_de_big_data_uni`.`cryptos`.`aggregated_crypto_prices`
WHERE crypto = "bitcoin"
;

Databricks visualization. Run in Databricks to view.

In [0]:
Materialized View  has been created.